# Domain Evaluation — Legal & Finance

Aegis ships with domain-specific benchmark suites for **Legal** (18 dimensions)
and **Finance** (20 dimensions). This notebook walks through running domain
benchmarks, inspecting cases, and comparing against baselines.

In [ ]:
from aegis import EvalConfig, Evaluator
from aegis.eval.dimensions.registry import DimensionRegistry

## 1. Explore Domain Dimensions

The dimension registry holds all 101+ dimensions. Let's filter by domain.

In [ ]:
registry = DimensionRegistry.instance()

for domain in ["legal", "finance"]:
    dims = registry.list_by_domain(domain)
    print(f"\n{domain.upper()} — {len(dims)} dimensions:")
    for d in dims:
        print(f"  [{d.tier}] {d.id:<40s} {d.name}")

## 2. Run a Legal Domain Evaluation

Filter the evaluator to legal dimensions only.

In [ ]:
legal_dims = [d.id for d in registry.list_by_domain("legal")]

legal_result = Evaluator(config=EvalConfig(
    dimensions=legal_dims,
    seed=42,
)).run()

print(f"Legal overall: {legal_result.overall_score:.2%}")
print(f"Dimensions:    {len(legal_result.dimension_scores)}")

for dim_id, score in sorted(legal_result.dimension_scores.items(), key=lambda kv: kv[1], reverse=True):
    bar = '\u2588' * int(score * 30)
    print(f"  {dim_id:<40s} {score:.4f} {bar}")

## 3. Run a Finance Domain Evaluation

In [ ]:
finance_dims = [d.id for d in registry.list_by_domain("finance")]

finance_result = Evaluator(config=EvalConfig(
    dimensions=finance_dims,
    seed=42,
)).run()

print(f"Finance overall: {finance_result.overall_score:.2%}")
print(f"Dimensions:      {len(finance_result.dimension_scores)}")

for dim_id, score in sorted(finance_result.dimension_scores.items(), key=lambda kv: kv[1], reverse=True):
    bar = '\u2588' * int(score * 30)
    print(f"  {dim_id:<40s} {score:.4f} {bar}")

## 4. Benchmark Suites

Aegis includes structured benchmark suites with curated test cases.
Let's load and run the legal memory benchmark.

In [ ]:
from aegis.eval.benchmarks import FinanceMemoryBenchmark, LegalMemoryBenchmark

legal_bench = LegalMemoryBenchmark()
legal_report = legal_bench.run()

print("Legal Memory Benchmark")
print(f"  Cases:          {legal_report.total_cases}")
print(f"  Mean score:     {legal_report.mean_score:.4f}")
print(f"  Std deviation:  {legal_report.std_score:.4f}")
print(f"  Pass rate:      {legal_report.pass_rate:.1%}")

In [ ]:
finance_bench = FinanceMemoryBenchmark()
finance_report = finance_bench.run()

print("Finance Memory Benchmark")
print(f"  Cases:          {finance_report.total_cases}")
print(f"  Mean score:     {finance_report.mean_score:.4f}")
print(f"  Std deviation:  {finance_report.std_score:.4f}")
print(f"  Pass rate:      {finance_report.pass_rate:.1%}")

## 5. Benchmark Harness

The `BenchmarkHarness` can load any registered suite and run it.

In [ ]:
import sys

sys.path.insert(0, "..")

from benchmarks.harness import BenchmarkHarness

harness = BenchmarkHarness()

# Load and inspect the core suite
core_cases = harness.load_suite("core")
print(f"Core suite: {len(core_cases)} cases")
for case in core_cases[:5]:
    print(f"  [{case.tier.value}] {case.dimension_id}: {case.prompt[:60]}...")

# Run with the harness
results = harness.run("core")
print("\nCore benchmark results:")
print(f"  Mean score: {results['mean_score']:.4f}")
print(f"  Cases run:  {results['total_cases']}")

## 6. Cross-Domain Comparison

Compare legal vs finance performance side-by-side.

In [ ]:

# Run a combined eval
all_domain_dims = legal_dims + finance_dims
combined = Evaluator(config=EvalConfig(dimensions=all_domain_dims, seed=42)).run()

# Split by domain
legal_set = set(legal_dims)
finance_set = set(finance_dims)

legal_scores = [s for d, s in combined.dimension_scores.items() if d in legal_set]
finance_scores = [s for d, s in combined.dimension_scores.items() if d in finance_set]

print("Cross-Domain Comparison")
print(f"  Legal:   mean={sum(legal_scores)/len(legal_scores):.4f}  dims={len(legal_scores)}")
print(f"  Finance: mean={sum(finance_scores)/len(finance_scores):.4f}  dims={len(finance_scores)}")
print(f"  Overall: {combined.overall_score:.4f}")

## Next Steps

- Add custom domain dimensions with `DimensionRegistry.register()`
- Create benchmark suites by subclassing `BenchmarkSuite`
- Run via CLI: `aegis eval benchmark --suite legal`
- See `03_memory_operations.ipynb` for memory infrastructure
- See `04_ingestion_pipeline.ipynb` for document ingestion